In [ ]:
from modelscope.hub.snapshot_download import snapshot_download

# load model (only done first time)
model_dir = snapshot_download('iic/nlp_raner_named-entity-recognition_chinese-base-generic', cache_dir='/scratch/ssd004/scratch/cambish/CNER')

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.models import Model

ner_model = Model.from_pretrained(model_dir)
ner_pipeline = pipeline(Tasks.named_entity_recognition, model=ner_model)

# test sample
ner_input = ['尼克松在回顾1972年访华时说：“我知道，1954年在……时福斯特·杜勒斯（当时的美国国务卿）拒绝同周（周恩来）握手，使他身受侮辱。因此我走完梯级时决心伸出我的手，一边向他走去。当我们的手相握的时候，一个时代结束了，另一个时代开始了。”对上述材料理解正确的是',
            '尼克松与周恩来握手意味着美国彻底放弃了遏制中国政策', '尼克松否定了福斯特·杜勒斯的对华态度', '中美关系对20世纪70年代的国际关系有重要影响', '省略部分应是万隆会议']
outputs = ner_pipeline(ner_input)
print(outputs)

In [ ]:
for output in outputs:
    print(output['output'])

In [ ]:
import spacy
from spacy.tokens import DocBin
from spacy import displacy

nlp = spacy.blank("zh")

# docs list will store annotations
docs = []
for text, annotations in zip(ner_input, outputs):
    doc = nlp(text)
    ents = []
    # required to convert annotations dict of lists into annotation dict
    for annotation in annotations['output']:
        span = doc.char_span(annotation['start'], annotation['end'], label=annotation['type'])
        ents.append(span)
    doc.ents = ents
    docs.append(doc)

displacy.render(docs, style="ent")